In [1]:
%matplotlib inline
import chainer
import matplotlib.pyplot as plt
import net_2
import numpy as np
import os
import chainer.functions as F
from train_vae_with_specified_label import *

In [2]:
# set paths and paramters
out_dir_path = './result'
log_path = os.path.join(out_dir_path, 'log')
model_name = 'model_100.npz'
dimz = 100

In [3]:
# load a model
model = net_2.VAE(784, dimz, 500, F.softplus) 
chainer.serializers.load_npz(os.path.join(out_dir_path, model_name), model, strict=True)
model.to_cpu()

KeyError: 'ld3/W is not a file in the archive'

In [ ]:
# load dataset
# src_train, src_test = chainer.datasets.get_mnist(withlabel=True) 
# use binarized mnist
TRAIN_PATH = '/root/data/binarized_mnist/train.npy'
TEST_PATH = '/root/data/binarized_mnist/test.npy'
src_train = np.load(TRAIN_PATH)
src_test = np.load(TEST_PATH) 

In [ ]:
# extract specified labels
specified_label = 0
train = extract_specified_labels(src_train, specified_label)
test = extract_specified_labels(src_test, specified_label)
print(train.shape)
print(test.shape)

In [ ]:
def save_images(x, filename):
    fig, ax = plt.subplots(3, 3, figsize=(9, 9), dpi=100)
    for ai, xi in zip(ax.flatten(), x):
        ai.imshow(xi.reshape(28, 28))
    fig.savefig(filename)

In [ ]:
# encode and decode training dataset
train_ind = [1, 3, 5, 10, 2, 0, 13, 15, 17]  
x = chainer.Variable(np.asarray(train[train_ind])) 
with chainer.using_config('train', False), chainer.no_backprop_mode(): 
    x1 = model(x) 

In [ ]:
# display and save images
print('train')
save_images(x.data, os.path.join(out_dir_path, 'train')) 

In [ ]:
print('train_reconstructed')
save_images(x1.data, os.path.join(out_dir_path, 'train_reconstructed'))  

In [ ]:
# see difference between them
dx = np.abs(x.data - x1.data)
save_images(dx, os.path.join(out_dir_path, 'train_diff')) 

In [ ]:
# encode and decode test dataset
test_ind = [3, 2, 1, 18, 4, 8, 11, 17, 61]
x = chainer.Variable(np.asarray(test[test_ind]))
with chainer.using_config('train', False), chainer.no_backprop_mode():
    x1 = model(x)

In [ ]:
print('test')    
save_images(x.data, os.path.join(out_dir_path, 'test'))

In [ ]:
print('test reconstructed')
save_images(x1.data, os.path.join(out_dir_path, 'test_reconstructed'))

In [ ]:
# see difference between them
dx = np.abs(x.data - x1.data)
save_images(dx, os.path.join(out_dir_path, 'test_diff')) 

In [ ]:
# draw images from randomly sampled z
np.random.seed(1234)
z = chainer.Variable(
    np.random.normal(0, 1, (9, dimz)).astype(np.float32))
x = model.decode(z)
save_images(x.data, os.path.join(out_dir_path, 'sampled'))

In [ ]:
def load_loss(path):
    return [float(line) for line in open(path)]

import re
def loss_generator(path, s):
    s = re.compile(r'"{}": (-?[0-9]+.[0-9]+),?'.format(s))
    for line in open(path):
        m = s.findall(line)
        if m:
            yield float(m[0])

In [ ]:
# extract information from log
main_loss = list(loss_generator(log_path, 'main/loss'))
validation_main_loss = list(loss_generator(log_path, 'validation/main/loss'))
main_mu = list(loss_generator(log_path, 'main/mu'))
validation_main_mu = list(loss_generator(log_path, 'validation/main/mu'))
main_sigma = list(loss_generator(log_path, 'main/sigma'))
validation_main_sigma = list(loss_generator(log_path, 'validation/main/sigma'))

In [ ]:
# display loss
plt.figure(figsize=(10, 5))
plt.title('loss')
plt.xlabel('epoch', fontsize=20)
plt.ylabel('loss', fontsize=20)
plt.xlim(0, 100)
plt.ylim(60, 180)
plt.plot(main_loss, label='train')
plt.plot(validation_main_loss, label='test')
plt.legend(loc='best')
plt.show()

In [ ]:
# display mu
plt.figure(figsize=(10, 5))
plt.title('mu')
plt.xlabel('epoch', fontsize=20)
plt.ylabel('mu', fontsize=20)
plt.xlim(0, 100)
# plt.ylim(-0.01, 0.01)
plt.plot(main_mu, label='train')
plt.plot(validation_main_mu, label='test')
plt.legend(loc='best')
plt.show()

In [ ]:
# display sigma
plt.figure(figsize=(10, 5))
plt.title('sigma')
plt.xlabel('epoch', fontsize=20)
plt.ylabel('sigma', fontsize=20)
plt.xlim(0, 100)
# plt.ylim(8, 22)
plt.plot(main_sigma, label='train')
plt.plot(validation_main_sigma, label='test')
plt.legend(loc='best')
plt.show()

In [ ]:
def decode(dataset):
    ind = [1, 3, 5, 10, 2, 0, 13, 15, 17]  
    x = chainer.Variable(np.asarray(dataset[ind])) 
    with chainer.using_config('train', False), chainer.no_backprop_mode(): 
        mu, ln_var = model.encode(x) 
        average_mu = np.mean(mu.data)
        sigma = np.exp(ln_var.data/2)
        average_sigma = np.mean(sigma)
        return average_mu, average_sigma

In [ ]:
# average mu and sigma for training data
mu, sigma = decode(train)
print('mu(0): {}'.format(mu))
print('sigma(1): {}'.format(sigma))

In [ ]:
# average mu and sigma for test data
mu, sigma = decode(test)
print('mu(0): {}'.format(mu))
print('sigma(1): {}'.format(sigma))

# 画像の符号化

In [ ]:
# select 1,000 images with label zero, which are considered as normal images
zero_number = 1000
zeros = train[:zero_number]
zeros.shape

In [ ]:
# encode them
xs = chainer.Variable(zeros) 
with chainer.using_config('train', False), chainer.no_backprop_mode(): 
    zero_mus, zero_ln_vars = model.encode(xs) 

In [ ]:
# select 100 images with label six, which are considered as anormaly images
anomaly_label = 6
sixs = extract_specified_labels(src_train, anomaly_label)[:100]
sixs.shape

In [ ]:
# encode them
xs = chainer.Variable(sixs) 
with chainer.using_config('train', False), chainer.no_backprop_mode(): 
    six_mus, six_ln_vars = model.encode(xs) 

In [ ]:
# concatenate normal and anormaly mus
dataset = np.concatenate([zero_mus.data, six_mus.data], axis=0)
print(dataset.shape)

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
# reduce dimensions
reduced_dataset = TSNE(n_components=2, random_state=0).fit_transform(dataset)
# reduced_dataset = dataset.copy()

In [ ]:
reduced_dataset.shape

In [ ]:
# split data into normal and anormaly data
zero_reduced_dataset = reduced_dataset[:zero_number]
six_reduced_dataset = reduced_dataset[zero_number:]
zero_reduced_dataset.shape

In [ ]:
# calculate mahalanobis distance
from scipy.spatial.distance import mahalanobis

# calculate covariance matrix for training data
sigma = np.cov(zero_reduced_dataset, rowvar=False)

# make inverse of it
inv_sigma = np.linalg.inv(np.cov(zero_reduced_dataset, rowvar=False)) 

# calculate mean for training data
mean = np.mean(zero_reduced_dataset, axis=0)

# calculate mahalanobis distances
outlier_dists = [mahalanobis(mean, outlier, inv_sigma) for outlier in six_reduced_dataset]
inlier_dists = [mahalanobis(mean, inlier, inv_sigma) for inlier in zero_reduced_dataset]

average_outlier_dist = np.mean(outlier_dists)
average_inlier_dist = np.mean(inlier_dists)
print('average_outlier_dist', average_outlier_dist)
print('average_inlier_dist', average_inlier_dist)
print(sigma)

In [ ]:
# display 
plt.figure(figsize=(10, 10))
plt.scatter(reduced_dataset[:zero_number, 0], reduced_dataset[:zero_number, 1], c='blue', edgecolors='blue', label='zero', marker='o')
plt.scatter(reduced_dataset[zero_number:, 0], reduced_dataset[zero_number:, 1], c='green', edgecolors='green', label='six', marker='o')
plt.axis('equal')


n = 100
x = np.linspace(-60, 60)
y = np.linspace(-60, 60)
X, Y = np.meshgrid(x, y)
Z = np.empty((n//2, n//2))
for r  in range(n // 2):
    for c in range(n // 2):
        v = np.c_[X[r, c], Y[r, c]]
        Z[r, c] = mahalanobis(mean, v, inv_sigma)

plt.contour(X, Y, np.array(Z), levels=[1, 2, 3, 4])

plt.xlim(-60, 60)
plt.ylim(-60, 60)

plt.grid(True)
plt.legend(loc='best')
plt.show()

In [ ]:
# display F values
plt.figure(figsize=(10, 5))
thrs = []
fs = []
prs = []
res = []
with open('./f_value.txt') as fin:
    for line in fin:
        thr, f, pr, re = [float(item) for item in line.strip().split()]
        thrs.append(thr)
        fs.append(f)
        prs.append(pr)
        res.append(re)
plt.xlabel('threshold', fontsize=20)
plt.plot(thrs, res, marker='.', label='Recall')
plt.plot(thrs, fs, marker='.', label='F value')
plt.plot(thrs, prs, marker='.', label='Precision')
plt.legend(loc='best')
plt.show()

# 画像の符号化・復号化

In [ ]:
anomaly_label = 6
random_sixs = extract_specified_labels(src_train, anomaly_label)[:9]
random_sixs.shape

In [ ]:
x = chainer.Variable(random_sixs)
with chainer.using_config('train', False), chainer.no_backprop_mode():
    x1 = model(x)

In [ ]:
# 差をとる。
dx = np.abs(x.data - x1.data)
save_images(dx, os.path.join(out_dir_path, 'six_diff')) 